In [1]:
from typing import List, Dict, Set, Any, Optional, Tuple, Literal, Callable
import numpy as np
import torch
from torch import Tensor
import sigkernel
import os
import sys
import tslearn
import tslearn.metrics
import ksig

from kernels.abstract_base import TimeSeriesKernel, StaticKernel
from kernels.static_kernels import LinearKernel, RBFKernel, PolyKernel
from kernels.integral import StaticIntegralKernel
from kernels.sig_pde import SigPDEKernel
from kernels.sig_trunc import TruncSigKernel
from kernels.gak import GlobalAlignmentKernel, sigma_gak
from kernels.flattened_static import FlattenedStaticKernel
from kernels.reservoir import ReservoirKernel

In [ ]:
import ksig
import timeit

#### Test GAK ####
N= 80
N2= 500
T, d = 70, 2
torch.manual_seed(0)
X = torch.randn(N,  T, d, dtype=torch.float64).to("cuda") / d
Y = torch.randn(N2, T, d, dtype=torch.float64).to("cuda") / d
X_np = X.cpu().numpy()
Y_np = Y.cpu().numpy()

sigma = tslearn.metrics.sigma_gak(X_np)
gak = tslearn.metrics.cdist_gak
ksigker = ksig.kernels.GlobalAlignmentKernel(static_kernel=ksig.static.kernels.RBFKernel(bandwidth=sigma))
mine = GlobalAlignmentKernel(RBFKernel(sigma=sigma), normalize=True, max_batch=50000)
# ksigker = ksig.kernels.SignatureKernel(static_kernel=ksig.static.kernels.RBFKernel(bandwidth=sigma), n_levels=5, order=1)
# mine = TruncSigKernel(RBFKernel(sigma=sigma), normalize=False, trunc_level=5, geo_order=1, max_batch=50000)
# ksigker = ksig.kernels.SignatureKernel(static_kernel=ksig.static.kernels.LinearKernel(), n_levels=5, order=1)
# mine = TruncSigKernel(LinearKernel(), normalize=True, trunc_level=5, geo_order=1, max_batch=50000)


#out = gak(X, X, sigma=sigma)
out2 = ksigker(X_np, X_np)
print(out2)
out3 = mine(X, X)
#print(out)
print(out3)
print(np.mean(np.abs(out2 - out3.cpu().numpy())))

def function1():
    gak(X, X_np, sigma=sigma)

def function2():
    ksigker(X_np, X_np)

def function3():
    with torch.no_grad():
        mine(X, Y)

# # Measure the execution time of function 1
# execution_time1 = timeit.timeit(function1, number=1)

# Measure the execution time of function 2
execution_time2 = timeit.timeit(function2, number=1)
print("Execution time of function 2:", execution_time2)
# Measure the execution time of function 3
execution_time3 = timeit.timeit(function3, number=1)
print("Execution time of function 3:", execution_time3)

In [ ]:
#iterate over antidiagonals with s,t>0
T1 = 5
T2 = 4
for diag in range(2, T1+T2-1):
    for s in range(max(1, diag - T2 + 1), min(diag, T1)):
        t = diag - s
        print(s,t)
    print("\n")

In [ ]:
# test for loop indices
import time
import torch
import itertools
from kernels.static_kernels import LinearKernel

lin_ker = LinearKernel()

def placeholder_ker(X:Tensor, Y:Tensor, diag:bool=False):
    return lin_ker.time_gram(X, Y, diag)[...,0,0]


def test_indices(X:Tensor, 
                 Y:Tensor,
                 diag:bool,
                max_batch:int, 
    ):
    device = X.device
    N1, T, d = X.shape
    N2, _, _ = Y.shape

    # split into batches. FASTEST METHOD NO BATCH
    t1 = time.perf_counter()
    result = placeholder_ker(X, Y)
    t2 = time.perf_counter()
    print("time NOBATCH\t", t1-t2)

    # split into batches BY INDICES
    t1 = time.perf_counter()
    if diag:
        indices = torch.arange(N1, device=device).tile(2,1) # shape (2, N)
    else:
        indices = torch.cartesian_prod(torch.arange(N1, device=device), 
                                    torch.arange(N2, device=device)).T #shape (2, N1*N2)
    split = torch.split(indices, max_batch, dim=1)
    result = [placeholder_ker(X[ix], Y[iy], diag=True) for ix,iy in split]
    t2 = time.perf_counter()
    print("time INDEX\t", t1-t2)

    # split into batches VIA SPLIT
    t1 = time.perf_counter()
    split_X = torch.split(X, max_batch, dim=0)
    Y_max_batch = max(1, max_batch//N1)
    split_Y = torch.split(Y, Y_max_batch, dim=0)
    result = [placeholder_ker(ix, iy) for ix,iy in itertools.product(split_X, split_Y)]
    if max_batch >= N1:
        result = torch.cat(result, dim=1)
    else:
        result = torch.cat(result, dim=0).reshape(N1, N2)
    t2 = time.perf_counter()
    print("time SPLIT\t", t1-t2)

X = torch.randn(200, 7, 10)
Y = torch.randn(300, 7, 10)
test_indices(X, Y, False, 10000)

#split = torch.split(X, )

In [2]:
#### test dimensions of TimeSeriesKernels ####

N=3
N2=4
T, d = 7, 5
X = torch.randn(N, T, d) / d**0.5
Y = torch.randn(N2, T, d) / d**0.5
inputs = [
    (X, X),
    (X, Y),
    (X[0], X[0]),
    (X[0], Y[0]),
    (X[0], Y),
    (X, Y[0]),
]
diag_inputs = [
    (X, X),
    (Y, Y),
    (X[:min(N,N2)], Y[:min(N,N2)]),
    (X[0], X[0]),
    (X[0], Y[0]),
]
def test_kernel(ker: TimeSeriesKernel, inputs, diag=False):
    print(ker)
    for X, Y in inputs:
        out = ker(X, Y, diag, normalize=False)
        out_normalize = ker(X, Y, diag, normalize=True)
        # print(out, "out")
        # print(out_normalize, "out, normalize")
        print(out.shape)
    print()


sigker = TruncSigKernel(static_kernel=RBFKernel(), 
                        trunc_level=6, 
                        geo_order=1,
                        only_last=False,)
test_kernel(sigker, inputs)
test_kernel(sigker, diag_inputs, True)

sigpde = SigPDEKernel(static_kernel=RBFKernel(),
                     dyadic_order=3,)
test_kernel(sigpde, inputs)
test_kernel(sigpde, diag_inputs, True)

intker = StaticIntegralKernel(static_kernel=RBFKernel())
test_kernel(intker, inputs)
test_kernel(intker, diag_inputs, True)

pde = SigPDEKernel(static_kernel=RBFKernel(), dyadic_order=2)
test_kernel(pde, inputs)
test_kernel(pde, diag_inputs, True)

gak = GlobalAlignmentKernel(static_kernel=RBFKernel(sigma=sigma_gak(X)))
test_kernel(gak, inputs)
test_kernel(gak, diag_inputs, True)

flat = FlattenedStaticKernel(static_kernel=LinearKernel())
test_kernel(flat, inputs)
test_kernel(flat, diag_inputs, True)

res = ReservoirKernel()
test_kernel(res, inputs)
test_kernel(res, diag_inputs, True)

torch.Size([3, 3, 6])
torch.Size([3, 4, 6])
torch.Size([1, 1, 6])
torch.Size([1, 1, 6])
torch.Size([1, 4, 6])
torch.Size([3, 1, 6])

torch.Size([3, 6])
torch.Size([4, 6])
torch.Size([3, 6])
torch.Size([1, 6])
torch.Size([1, 6])

torch.Size([3, 3])
torch.Size([3, 4])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 4])
torch.Size([3, 1])

torch.Size([3])
torch.Size([4])
torch.Size([3])
torch.Size([1])
torch.Size([1])

torch.Size([3, 3])
torch.Size([3, 4])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 4])
torch.Size([3, 1])

torch.Size([3])
torch.Size([4])
torch.Size([3])
torch.Size([1])
torch.Size([1])

torch.Size([3, 3])
torch.Size([3, 4])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 4])
torch.Size([3, 1])

torch.Size([3])
torch.Size([4])
torch.Size([3])
torch.Size([1])
torch.Size([1])

torch.Size([3, 3])
torch.Size([3, 4])
torch.Size([1, 1])
torch.Size([1, 1])
torch.Size([1, 4])
torch.Size([3, 1])

torch.Size([3])
torch.Size([4])
torch.Size([3])
torch.Size([1])
torc

In [ ]:
import numpy as np
import ksig

# Number of signature levels to use.
n_levels = 5 

# Use the RBF kernel for vector-valued data as static (base) kernel.
static_kernel = ksig.static.kernels.RBFKernel() 

# Instantiate the signature kernel, which takes as input the static kernel.
n_levels = 5
order = 1
sig_kernel = ksig.kernels.SignatureKernel(n_levels=n_levels, order=order, static_kernel=static_kernel)

# Generate 10 sequences of length 50 with 5 channels.
n_seq, l_seq, n_feat = 10, 50, 5 
X = np.random.randn(n_seq, l_seq, n_feat)

# Sequence kernels take as input an array of sequences of ndim == 3,
# and work as a callable for computing the kernel matrix. 
K_XX = sig_kernel(X)  # K_XX has shape (10, 10).

# The diagonal kernel entries can also be computed.
K_X = sig_kernel(X, diag=True)  # K_X has shape (10,).

# Generate another array of 8 sequences of length 20 and 5 features.
n_seq2, l_seq2 = 8, 20
Y = np.random.randn(n_seq2, l_seq2, n_feat)

# Compute the kernel matrix between arrays X and Y.
K_XY = sig_kernel(X, Y)  # K_XY has shape (10, 8)
K_XY

In [ ]:
#Test that iisig gives the same result as mine
import iisignature
import numpy as np
import ksig

# Number of signature levels to use.
normalize=False
trunc_level = 5
geo_order = 5
N=2
N2= 2
T, d = 20, 2
torch.manual_seed(0)
X = torch.randn(N, T, d, dtype=torch.float64).to("cuda") / d
Y = torch.randn(N2, T, d, dtype=torch.float64).to("cuda") / d
X_np = X.cpu().numpy()
Y_np = Y.cpu().numpy()

ksigker = ksig.kernels.SignatureKernel(static_kernel=ksig.static.kernels.LinearKernel(), 
                                       normalize=normalize,
                                       n_levels=trunc_level, 
                                       order=geo_order)
mine = TruncSigKernel(LinearKernel(scale=1), 
                      normalize=normalize, 
                      trunc_level=trunc_level, 
                      geo_order=geo_order, 
                      max_batch=50000)

#test
out1 = ksigker(X_np, Y_np)
out2 = mine(X, Y)
featuresX = iisignature.sig(X_np, trunc_level)
featuresY = iisignature.sig(Y_np, trunc_level)
out3 = 1+np.dot(featuresX, featuresY.T)
print("ksig", out1)
print("\nmine", out2)
print("\niisig", out3)
print(np.mean(np.abs(out1 - out2.cpu().numpy())))
print(np.mean(np.abs(out1 - out3)))
print(np.mean(np.abs(out2.cpu().numpy() - out3)))